# 1. Importing Packages

In [23]:
import os
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision

In [24]:

#Upload The Following Files
from unet import build_unet
from customMetrics import dice_loss, dice_coef, iou

# 2. Setting The Environment

In [25]:
H = 128
W = 128

In [26]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Hyperparameters """
batch_size = 64

lr = 1e-4
num_epochs = 100
model_path = os.path.join("/content/files128", "model128_basic.h5")
csv_path = os.path.join("/content/files128", "data_basic.csv")

In [27]:
""" Dataset """
# Make The Folder and upload the data
dataset_path = "/content/drive/MyDrive/gamma_data_128"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "valid")

In [31]:
!rm -rf files128

In [32]:
!rm -rf logs

In [33]:
pwd

'/content'

# 3. Utils Function

In [34]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [35]:
def load_data(path):
    print(path)
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "gt", "*.jpg")))
    return x, y

In [36]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

In [37]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

In [38]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

In [39]:
def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

In [40]:
train_x, train_y = load_data(train_path)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

/content/drive/MyDrive/gamma_data_128/train
/content/drive/MyDrive/gamma_data_128/valid
Train: 1890 - 1890
Valid: 630 - 630


In [41]:
pwd

'/content'

In [42]:
create_dir("/content/files128")

In [43]:
train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

train_steps = len(train_x)//batch_size
valid_steps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
if len(valid_x) % batch_size != 0:
    valid_steps += 1

In [44]:
train_steps, valid_steps

(30, 10)

# 4. The Model

In [45]:
""" Model """
model = build_unet((H, W, 3))
model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 128, 128, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_18 (BatchNo (None, 128, 128, 64) 256         conv2d_19[0][0]                  
__________________________________________________________________________________________________
activation_18 (Activation)      (None, 128, 128, 64) 0           batch_normalization_18[0][0]     
_______________________________________________________________________________________________

In [46]:
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
]

In [47]:
   model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks
)

Epoch 1/100
30/30 [==============================] - 25s 640ms/step - loss: 0.4415 - dice_coef: 0.2036 - iou: 0.1136 - recall_1: 0.3301 - precision_1: 0.3705 - val_loss: 0.7051 - val_dice_coef: 0.1384 - val_iou: 0.0744 - val_recall_1: 0.3808 - val_precision_1: 0.1038

Epoch 00001: val_loss improved from inf to 0.70514, saving model to /content/files128/model128_basic.h5
Epoch 2/100
30/30 [==============================] - 15s 489ms/step - loss: 0.2573 - dice_coef: 0.2985 - iou: 0.1761 - recall_1: 0.2694 - precision_1: 0.8118 - val_loss: 0.6477 - val_dice_coef: 0.1350 - val_iou: 0.0724 - val_recall_1: 0.0147 - val_precision_1: 0.0135

Epoch 00002: val_loss improved from 0.70514 to 0.64767, saving model to /content/files128/model128_basic.h5
Epoch 3/100
30/30 [==============================] - 15s 490ms/step - loss: 0.1985 - dice_coef: 0.3887 - iou: 0.2419 - recall_1: 0.3251 - precision_1: 0.8465 - val_loss: 0.5350 - val_dice_coef: 0.1258 - val_iou: 0.0671 - val_recall_1: 1.2864e-04 - va